# Pruebas RAG

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### 1️⃣ Preparar la Base de Datos de Sesgos
Este paso convierte frases con sesgos en vectores y las almacena en FAISS.

In [17]:
# Cargar modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Base de datos con ejemplos de sesgo
bias_examples = [
    "Las mujeres son emocionales y no aptas para liderar.",
    "El hombre promedio es más racional que una mujer.",
    "El hombre ha logrado avances científicos impresionantes.",  # Androcentrismo
    "La zorra es astuta, pero el zorro es valiente.",  # Uso dual aparente
    "La doctora María Pérez"  # Uso de cargo para referirse a mujeres
]

# Convertir en embeddings y almacenar en FAISS
bias_embeddings = np.array([embedding_model.encode(text) for text in bias_examples])
index = faiss.IndexFlatL2(bias_embeddings.shape[1])
index.add(bias_embeddings)

### 2️⃣ Buscar Sesgos en un Texto Nuevo
Cuando entra un texto, buscamos frases similares en la base de datos.

In [26]:
def retrieve_relevant_examples(query, k=2):
    """ Busca en FAISS los ejemplos más similares al texto dado """
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    _, idxs = index.search(query_embedding, k)
    return [bias_examples[i] for i in idxs[0]]

In [29]:
# Texto a analizar
new_text = "Las jugadoras chicas han realizado un gran partido."

In [30]:
# Recuperar ejemplos similares desde FAISS
retrieved_examples = retrieve_relevant_examples(new_text)
print("Ejemplos recuperados:", retrieved_examples)

Ejemplos recuperados: ['El hombre promedio es más racional que una mujer.', 'Las mujeres son emocionales y no aptas para liderar.']


### 3️⃣ Generar el Prompt y Pasarlo a LLM
Integramos la información recuperada en un prompt y pedimos a un LLM que analice el texto.

#### Salamandra

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Cargar Salamandra-2B
model_name = "BSC-LT/salamandra-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [38]:
# Construir el prompt con los ejemplos recuperados
prompt = f"""Eres un experto en lenguaje inclusivo y sesgos de género. Analiza el siguiente texto:

Ejemplos de sesgos detectados:
{retrieved_examples}

Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato:

Ejemplo de respuesta:
Lenguaje sexista: 1 (La frase implica que solo los hombres han conquistado el espacio, excluyendo a las mujeres).
Uso de 'hombre' para denominar a la humanidad: 1 (Se usa 'hombre' para referirse a la humanidad en general, en lugar de términos neutros como 'humanidad').
Uso dual aparente: 0
Uso de cargos/profesiones para mujeres: 0
Sexismo social: 0
Androcentrismo: 1 (Se centra en los logros masculinos sin mencionar el papel de las mujeres).
Asimetría en tratamiento: 0
Infantilización de mujeres: 0
Denominación sexualizada: 0
Denominación redundante: 0
Denominación dependiente: 0

Ahora, analiza el siguiente texto y responde siguiendo el formato anterior:

Texto a analizar: "{new_text}"

Respuesta:"""

In [40]:
# Tokenizar e inferir
inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    output = model.generate(**inputs,
                            max_new_tokens=200,
                            do_sample=True,
                            # temperature=0.1
                            )

# Decodificar la respuesta
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nRespuesta de Salamandra:")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Respuesta de Salamandra:
Eres un experto en lenguaje inclusivo y sesgos de género. Analiza el siguiente texto:

Ejemplos de sesgos detectados:
['El hombre promedio es más racional que una mujer.', 'Las mujeres son emocionales y no aptas para liderar.']

Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato:

Ejemplo de respuesta:
Lenguaje sexista: 1 (La frase implica que solo los hombres han conquistado el espacio, excluyendo a las mujeres).
Uso de 'hombre' para denominar a la humanidad: 1 (Se usa 'hombre' para referirse a la humanidad en general, en lugar de términos neutros como 'humanidad').
Uso dual aparente: 0
Uso de cargos/profesiones para mujeres: 0
Sexismo social: 0
Androcentrismo: 1 (Se centra en los logros masculinos sin mencionar el papel de las mujeres).
Asimetría en tratamiento: 0
Infantilización de mujeres: 0
Denominación sexualizada: 0
Denominación redundante: 0
Denominaci

#### OpenAI

In [46]:
import configparser
import openai
from pydantic import BaseModel
import json
from tqdm import tqdm
import pandas as pd

In [ ]:
class LenguajeSexista(BaseModel):
    etiqueta: int
    explicacion: str

class HombreHumanidad(BaseModel):
    etiqueta: int
    explicacion: str

class DualAparente(BaseModel):
    etiqueta: int
    explicacion: str

class CargosMujeres(BaseModel):
    etiqueta: int
    explicacion: str

class SexismoSocial(BaseModel):
    etiqueta: int
    explicacion: str

class Androcentrismo(BaseModel):
    etiqueta: int
    explicación: str

class Asimetria(BaseModel):
    etiqueta: int
    explicacion: str

class Infantilizacion(BaseModel):
    etiqueta: int
    explicacion: str

class DenominacionSexualizada(BaseModel):
    etiqueta: int
    explicacion: str

class DenominacionRedundante(BaseModel):
    etiqueta: int
    explicacion: str

class DenominacionDependiente(BaseModel):
    etiqueta: int
    explicacion: str

class ResponseFormat(BaseModel):
    lenguaje_sexista: LenguajeSexista
    hombre_humanidad: HombreHumanidad
    dual_aparente: DualAparente
    cargos_mujeres: CargosMujeres
    sexismo_social: SexismoSocial
    androcentrismo: Androcentrismo
    asimetria: Asimetria
    infantilizacion: Infantilizacion
    denominacion_sexualizada: DenominacionSexualizada
    denominacion_redundante: DenominacionRedundante
    denominacion_dependiente: DenominacionDependiente

config = configparser.ConfigParser()
config.read('../pruebas_openai/config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']

In [56]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data = data.iloc[[9, 10]]

In [57]:
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
9,9,129,129,https://www.infolibre.es/cultura/esther-vivas-...,1,2019,20/3/2019,Clara Morales,Clara Morales,1,...,,,,,,,,,,"Dice Esther Vivas (Sabadell, 1975) que ""la lit..."
10,10,133,133,https://www.infolibre.es/politica/cientificas-...,1,2019,6/2/2019,Europa Press,Agencia /otros medios,4,...,,,,,,,,,,Científicas han debatido este miércoles sobre ...


In [58]:
predictions = []

# Loop de predicción
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Procesando predicciones"):
    n = row['N']
    n_periodico = row['n_periodico']
    contenido = row['contenido']
    try:
        # Llamada al modelo para detectar las citas humanas y sus fuentes
        completion = openai.beta.chat.completions.parse(
            model=OPENAI_MODEL,
            messages=[
                {
                    "role": "system", 
                    "content": """
                                    Eres un experto en lenguaje inclusivo y sesgos de género. 
                                    Tu tarea es analizar textos para detectar diferentes tipos de sesgos de género y lenguaje sexista. 
                                    Clasifica el texto a analizar según los criterios definidos y proporciona una breve explicación para cada categoría detectada. 
                                    El formato de la respuesta debe estar estructurado de acuerdo con las categorías y clases definidas.
                                """ 
                },
                {
                    "role": "user", 
                    "content": f"""
                                    Ejemplos de sesgos detectados:
                                    {retrieved_examples}

                                    Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato, asegurándote de incluir tanto la etiqueta como una breve explicación para cada categoría:

                                    Ejemplo de respuesta:
                                    {{
                                        "lenguaje_sexista": {{
                                            "etiqueta": 1,
                                            "explicacion": "La frase implica que solo los hombres han conquistado el espacio, excluyendo a las mujeres."
                                        }},
                                        "hombre_humanidad": {{
                                            "etiqueta": 1,
                                            "explicacion": "Se usa 'hombre' para referirse a la humanidad en general, en lugar de términos neutros como 'humanidad'."
                                        }},
                                        "dual_aparente": {{
                                            "etiqueta": 0,
                                            "explicacion": "No se observa un uso dual aparente."
                                        }},
                                        "cargos_mujeres": {{
                                            "etiqueta": 0,
                                            "explicacion": "No se utiliza un cargo o profesión para referirse a una mujer."
                                        }},
                                        "sexismo_social": {{
                                            "etiqueta": 0,
                                            "explicacion": "El texto no muestra sexismo social."
                                        }},
                                        "androcentrismo": {{
                                            "etiqueta": 1,
                                            "explicacion": "El texto se centra en los logros de los hombres sin mencionar el papel de las mujeres."
                                        }},
                                        "asimetria": {{
                                            "etiqueta": 0,
                                            "explicacion": "No hay asimetría en el tratamiento entre hombres y mujeres."
                                        }},
                                        "infantilizacion": {{
                                            "etiqueta": 0,
                                            "explicacion": "No se observa infantilización de las mujeres."
                                        }},
                                        "denominacion_sexualizada": {{
                                            "etiqueta": 0,
                                            "explicacion": "No hay denominación sexualizada."
                                        }},
                                        "denominacion_redundante": {{
                                            "etiqueta": 0,
                                            "explicacion": "No se usa una denominación redundante."
                                        }},
                                        "denominacion_dependiente": {{
                                            "etiqueta": 0,
                                            "explicacion": "No se observa denominación dependiente."
                                        }}
                                    }}

                                    Ahora, analiza el siguiente texto y responde siguiendo el formato anterior:

                                    Texto a analizar: "{contenido}"
                                """
                                                }
            ],
            response_format=ResponseFormat
        )
        
        # Extraer la respuesta
        response_message = completion.choices[0].message.content
        print(response_message)

        # Usamos json.loads para convertir el string JSON a un objeto Python
        response_data = json.loads(response_message)

        # Agregar el índice de la noticia al JSON
        response_data['indice'] = idx  # Agrega el índice al diccionario de cada cita
        response_data['N'] = n
        response_data['n_periodico'] = n_periodico
        
        # Agregar el resultado con el índice a las predicciones
        predictions.append(response_data)

    except Exception as e:
        print(f"Error en el índice {idx}: {e}")

Procesando predicciones:  50%|█████     | 1/2 [00:13<00:13, 13.51s/it]

{
    "lenguaje_sexista": {
        "etiqueta": 0,
        "explicacion": "El texto no utiliza un lenguaje denigrante o despectivo hacia las mujeres ni presenta actitudes que refuercen la discriminación de género."
    },
    "hombre_humanidad": {
        "etiqueta": 0,
        "explicacion": "No se hace uso de 'hombre' como un término que englobe a toda la humanidad, se menciona el término 'mujer' de manera explícita y sin exclusiones."
    },
    "dual_aparente": {
        "etiqueta": 1,
        "explicacion": "El texto presenta un uso de términos que, a primera vista, podrían considerarse neutros pero que al analizar el contexto, se observa que refuerzan la visión de que la maternidad es un tema exclusivamente femenino, sin considerar la implicación de los padres."
    },
    "cargos_mujeres": {
        "etiqueta": 0,
        "explicacion": "No se mencionan cargos o posiciones que indiquen un sesgo en la representación del trabajo de las mujeres."
    },
    "sexismo_social": {
    

Procesando predicciones: 100%|██████████| 2/2 [00:28<00:00, 14.17s/it]

{
    "lenguaje_sexista": {
        "etiqueta": 0,
        "explicacion": "El texto utiliza un lenguaje inclusivo y se enfoca en la equidad de género, sin recurrir a expresiones que denoten un sexismo explícito."
    },
    "hombre_humanidad": {
        "etiqueta": 0,
        "explicacion": "No se observa el uso del término 'hombre' para referirse a la humanidad en general, sino que se hace mención directa a mujeres y hombres en contextos relevantes."
    },
    "dual_aparente": {
        "etiqueta": 0,
        "explicacion": "El texto no cae en la dualidad aparente; se aboga explícitamente por la igualdad entre mujeres y hombres en la ciencia."
    },
    "cargos_mujeres": {
        "etiqueta": 1,
        "explicacion": "Se mencionan cargos como 'directora del CNIO' y 'doctoranda', reconociendo explícitamente la ocupación de mujeres en posiciones de liderazgo y especialización."
    },
    "sexismo_social": {
        "etiqueta": 1,
        "explicacion": "El texto aborda las desiguald

In [59]:
# Guardar las predicciones en un archivo JSON
with open('predictions_lenguaje.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)